In [ ]:
# Basic environment modules
import numpy as np
import pandas as pd
import psycopg2 as pg # PostgreSQL module
import time
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import style

# Data analysis modules
from CoolProp.CoolProp import PropsSI
from sklearn.preprocessing import MinMaxScaler, QuantileTransformer, PolynomialFeatures
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
from scipy.integrate import ode

# User-defined libraries
from Model import *
from PhysicalProperty import *
from StructuredQuery import *
from Numeric import *
from Authentication import *

print("Import modules successfully.")

In [ ]:
#############################################################
###################### CHF 계산 과정 ############################
#############################################################

sql = StructuredQuery()
pro = PhysicalProperty()
mod = Model()
oau = Authentication()

In [ ]:
# Create log table 
log_tb = pd.DataFrame()

# how to import the logs about nth steps for calculating steam quaility and calculated critical heat flux values. 

In [ ]:
# Connect to DB Server and get data
# Load json file (Security)
json_parse = oau.get_apikey(json_filename='key.json')
json_res = pd.json_normalize(json_parse['sql'])

# Set query from DB
sql = "SELECT * FROM CHF.raw_database WHERE refri = 'R12'" 

# Connect DB server
#(conn, db_engine) = sql.connect(json_res["host"][1], json_res["dbname"][1], json_res["user"][1], json_res["port"][1], json_res["password"][1], json_res["service"][1]) #postgreSQL
connect = pys.connect(host=json_res["host"][1], user = json_res["user"][1], password = json_res["password"][1], cursorclass=pys.cursors.DictCursor)
cur = connect.cursor()
cur.execute(sql)


result = cur.fetchall()
raw_tb = pd.DataFrame(result)

cur.close()
del result # delete temp file
del sql # delete quiry log

print("Loading database completed. data size: {}".format(len(raw_tb)))

In [ ]:
# Rawdata에서 Physical property 추가 (for water)
# if physical properties can't calculated, the other step is need (interpolation based on the physical properties table) = Sodium
prop_tb = raw_tb.copy()
"""
R113, R114 can't use viscosity model -> re-mapping these models based on interpolation tables
R12 Pressure range : 0.242551~4.1361e+06 Pa (P, Q Flash)
H2O Pressure range : 611.655~2.2064e+07 Pa (P, Q Flash)

"""

for i in range(0, len(raw_tb)):
    # calculated physical properties at saturation point
    prop_tb.loc[i,'tsat'] = round(PropsSI('T', 'P', raw_tb.loc[i, 'p'] * 1e5, 'Q', 0, raw_tb.loc[i, 'refri']),12)
    prop_tb.loc[i,'pcrit'] = round(PropsSI(raw_tb.loc[i, 'refri'],'pcrit') * 10**-5, 12)
    prop_tb.loc[i,'rdcp'] = round(raw_tb.loc[i,'p']/prop_tb.loc[i,'pcrit'],12)
    prop_tb.loc[i,'rhof'] = round(PropsSI('D', 'P', raw_tb.loc[i, 'p'] * 1e5, 'Q', 0, raw_tb.loc[i, 'refri']),12)
    prop_tb.loc[i,'rhov'] = round(PropsSI('D', 'P', raw_tb.loc[i, 'p'] * 1e5, 'Q', 1, raw_tb.loc[i, 'refri']),12)
    prop_tb.loc[i,'muf'] = round(PropsSI('V', 'P', raw_tb.loc[i, 'p'] * 1e5, 'Q', 0, raw_tb.loc[i, 'refri']),12)
    prop_tb.loc[i,'muv'] = round(PropsSI('V', 'P', raw_tb.loc[i, 'p'] * 1e5, 'Q', 1, raw_tb.loc[i, 'refri']),12)
    prop_tb.loc[i,'hfo'] = round(PropsSI('H', 'P', raw_tb.loc[i, 'p'] * 1e5, 'Q', 0, raw_tb.loc[i, 'refri'])*1e-3,12) #[kJ/kg]
    prop_tb.loc[i,'hvo'] = round(PropsSI('H', 'P', raw_tb.loc[i, 'p'] * 1e5, 'Q', 1, raw_tb.loc[i, 'refri'])*1e-3,12) #[kJ/kg]
    prop_tb.loc[i, 'lam'] = round((prop_tb.loc[i, 'hvo'] - prop_tb.loc[i,'hfo']),12) # [kJ/kg]
    if pd.isna(raw_tb.loc[i, 'v']):
        prop_tb.loc[i, 'v']     = round(raw_tb.loc[i, 'g'] / prop_tb.loc[i, 'rhof'], 12) # [m/s]
    else:
        prop_tb.loc[i, 'v'] = round(raw_tb.loc[i, 'v']) # [m/s]
    prop_tb.loc[i, 'cpf']   = round(PropsSI('C', 'P', raw_tb.loc[i, 'p'] * 1e5, 'Q', 0, raw_tb.loc[i, 'refri']),12) # [J/kgK]
    prop_tb.loc[i, 'cpv']   = round(PropsSI('C', 'P', raw_tb.loc[i, 'p'] * 1e5, 'Q', 1, raw_tb.loc[i, 'refri']),12) # [J/kgK]
    prop_tb.loc[i, 'sigma'] = round(PropsSI('I', 'P', raw_tb.loc[i, 'p'] * 1e5, 'Q', 0, raw_tb.loc[i, 'refri']),12) # [N/m]
    prop_tb.loc[i, 'kf'] = round(PropsSI('L', 'P', raw_tb.loc[i, 'p'] * 1e5, 'Q', 0, raw_tb.loc[i, 'refri']),12) # [W/m/K]
    prop_tb.loc[i, 'kv'] = round(PropsSI('L', 'P', raw_tb.loc[i, 'p'] * 1e5, 'Q', 1, raw_tb.loc[i, 'refri']),12) # [W/m/K]

    # enthin is empty
    # Fill Na related to inlet subcooling
    if np.isnan(raw_tb.loc[i, 'enthin']):
        if np.isnan(raw_tb.loc[i, 'tin']):
            # enthin, tin is empty
            if np.isnan(raw_tb.loc[i, 'xi']):
                # enthin, tin, and xi is empty
                prop_tb.loc[i, 'hin'] = 9999
                prop_tb.loc[i, 'enthin'] = prop_tb.loc[i, 'hfo'] - prop_tb.loc[i, 'hin'] # [kJ/kg]
                print("Error")
            else:
                # hin calculate
                prop_tb.loc[i, 'hin'] = round(prop_tb.loc[i, 'hfo'] - raw_tb.loc[i, 'xi']*prop_tb.loc[i, 'lam'], 12)
                prop_tb.loc[i, 'enthin'] = prop_tb.loc[i, 'hfo'] - prop_tb.loc[i, 'hin'] # [kJ/kg]
        else:
            prop_tb.loc[i, 'hin'] = round(PropsSI('H', 'P', raw_tb.loc[i, 'p'] * 1e5, 'T', raw_tb.loc[i, 'tin']+273.15, raw_tb.loc[i, 'refri']),12) # calculate inlet enthalpy using tin
            prop_tb.loc[i, 'enthin'] = prop_tb.loc[i, 'hfo'] - prop_tb.loc[i, 'hin'] # [kJ/kg]
    else:
        prop_tb.loc[i, 'hin'] = round(prop_tb.loc[i, 'hfo'] - raw_tb.loc[i, 'enthin'], 12) # [kJ/kg]
        prop_tb.loc[i, 'enthin'] = prop_tb.loc[i, 'hfo'] - prop_tb.loc[i, 'hin'] # [kJ/kg]
    
    prop_tb.loc[i, 'xi'] = -prop_tb.loc[i, 'enthin'] / prop_tb.loc[i, 'lam'] #[-]

    # xe is empty
    # Fill Na() related to outlet thermodynamic quality
    if pd.isna(raw_tb.loc[i, 'xe']):
        if pd.isna(raw_tb.loc[i, 'enthout']):
            # xe and enthout is empty
            prop_tb.loc[i, 'xe'] = round(pro.cal_xe(raw_tb.loc[i, 'q'], raw_tb.loc[i, 'doi'], raw_tb.loc[i, 'dio'], raw_tb.loc[i, 'geo'], raw_tb.loc[i, 'hs'], raw_tb.loc[i, 'g'], raw_tb.loc[i, 'enthin'], raw_tb.loc[i, 'lh'], prop_tb.loc[i, 'lam']), 12)
            prop_tb.loc[i, 'enthout'] = round(raw_tb.loc[i, 'xe'] * prop_tb.loc[i, 'lam'], 12)  #[-]
        else:
            prop_tb.loc[i, 'xe'] = round(-raw_tb.loc[i, 'enthout']/prop_tb.loc[i, 'lam'], 12)  #[-]
            prop_tb.loc[i, 'enthout'] = round(raw_tb.loc[i, 'enthout'], 12) # Copying raw data  #[-]
    else:
        prop_tb.loc[i, 'xe'] = round(raw_tb.loc[i, 'xe'], 12) # Copying raw data  #[-]
        prop_tb.loc[i, 'enthout'] = round(raw_tb.loc[i, 'xe'] * prop_tb.loc[i, 'lam'], 12)  #[-]
        
    #print("{} th data is complete to calculate.".format(i))
print("chf_tb에 Physical property 계산 완료")

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('mysql+pymysql://{}:{}@{}:{}/{}'.format(json_res["user"][1],json_res["password"][1],json_res["host"][1],json_res["port"][1],"CHF"), echo=False, encoding='utf-8')

# 계산된 osvTb의 properties 데이터를 PostgreSQL로 옮기기
#osvTb.index.name = 'index'
prop_tb.to_sql('prop_chf_tb', engine, if_exists='replace')

# correlation에대한 RMSE결과를 PostgreSQL로 이동
#corOSVTb.to_sql("rmse_osv_tb", engine, if_exists='replace')
#valOSVTb.to_sql("res_osv_tb", engine, if_exists='replace')
del engine

In [ ]:
for i, row in prop_tb.iterrows():
    # 무차원 수 계산하기
    prop_tb.loc[i, 'pe'] = round(
        pro.cal_pe(prop_tb.loc[i, 'dh'], prop_tb.loc[i, 'g'], prop_tb.loc[i, 'cpf'], prop_tb.loc[i, 'kf']), 6)
    prop_tb.loc[i, 're'] = round(pro.cal_re(prop_tb.loc[i, 'g'], prop_tb.loc[i, 'dh'], prop_tb.loc[i, 'muf']), 6)
    prop_tb.loc[i, 'we'] = round(
        pro.cal_we(prop_tb.loc[i, 'rhof'], prop_tb.loc[i, 'v'], prop_tb.loc[i, 'dh'], prop_tb.loc[i, 'sigma']), 6)
    prop_tb.loc[i, 'bd'] = round(
        pro.cal_bd(prop_tb.loc[i, 'rhof'], prop_tb.loc[i, 'rhov'], prop_tb.loc[i, 'dh'], prop_tb.loc[i, 'sigma']), 6)
    prop_tb.loc[i, 'pr'] = round(pro.cal_pr(prop_tb.loc[i, 'cpf'], prop_tb.loc[i, 'muf'], prop_tb.loc[i, 'kf']), 6)
    prop_tb.loc[i, 'ca'] = round(
        pro.cal_ca(prop_tb.loc[i, 'muf'], prop_tb.loc[i, 'v'], prop_tb.loc[i, 'sigma'], prop_tb.loc[i, 'rhof']), 6)
    prop_tb.loc[i, 'lc'] = np.sqrt(prop_tb.loc[i, 'sigma']/(9.8*(prop_tb.loc[i, 'rhof']-prop_tb.loc[i, 'rhov'])))

    if prop_tb.loc[i, 'geo'] == 'A':
        prop_tb.loc[i, 'dl'] = prop_tb.loc[i, 'dh']-2*prop_tb.loc[i, 'lc']
    elif prop_tb.loc[i, 'geo'] == 'R':
        if prop_tb.loc[i, 'dio'] < prop_tb.loc[i, 'lc']:
            prop_tb.loc[i, 'dl'] = prop_tb.loc[i, 'dh']/prop_tb.loc[i, 'ar']
        else:
            prop_tb.loc[i, 'dl'] = ((2*prop_tb.loc[i, 'doi']*(prop_tb.loc[i, 'dio'] - prop_tb.loc[i, 'lc']))/(prop_tb.loc[i, 'doi']+prop_tb.loc[i, 'dio'] - prop_tb.loc[i, 'lc']))/prop_tb.loc[i, 'ar']
    else:
        prop_tb.loc[i, 'dl'] = prop_tb.loc[i, 'dh']+2*prop_tb.loc[i, 'lc']

print("prop_tb에 대한 기본적인 계산 알고리즘을 완료하였습니다.")
print("corprop_tb에 대한 신규 알고리즘을 진행합니다.")

# OSV 모델 계산하기 (신규 테이블 corprop_tb)
#corprop_tb = pd.DataFrame(data=prop_tb[['source', 'run_id', 'xosv', 'tosv', 'xeq1', 'geo']])  # comparison table
#print("선택된 corprop_tb의 데이터 개수는 {}입니다.".format(len(corprop_tb)))

In [ ]:
# OSV, CHF 설정 모델 삽입 문구
"""
print('OSV correlation을 선택하시오: ')
print('(1) Saha and Zuber  (2) Modified Saha and Zuber  (3) Unal')
print('(4) Levy  (5) Bowring  (6) Jeong')
m_idx = int(input('-----너의 선택은? : '))
print("{}을 선택했습니다.".format(m_idx))

print('계산 방식을 선택하시오: ')
print('(1) Gauss-Jordan method  (2) exp_Heat_flux HW  (3) cal_Heat_flux HW (4) Bisection (Kim)')
cal_idx = int(input('-----너의 선택은? : '))
"""
m_idx = 1
cal_idx = 3

In [ ]:
# 결과테이블 새로 생성
res_tb = pd.DataFrame()
print('END TIME of INITIALIZATION : ',str(datetime.now())[10:19] )

# 모델별로 XOSV를 계산하기 위한 알고리즘
# def sub_xt_return(p, pcrit, dh, g, lam, m_idx):
if m_idx == 1:
    nm = 'sz'
    for i, row in prop_tb.iterrows():
        prop_tb.loc[i, 'dt_sz'], prop_tb.loc[i, 'x_sz'] = mod.cal_SZ(raw_tb.loc[i, 'q'], prop_tb.loc[i, 'rhof'], raw_tb.loc[i, 'dh'], raw_tb.loc[i, 'g'], 
                                               prop_tb.loc[i, 'cpf'], prop_tb.loc[i, 'kf'], prop_tb.loc[i, 'pe'], prop_tb.loc[i, 'lam']/1e-3)
        res_tb.loc[i,'st_cal'] = raw_tb.loc[i,'q']* 10 ** 6 / (raw_tb.loc[i,'g'] * prop_tb.loc[i,'cpf'] * prop_tb.loc[i,'dt_sz'])
    print("Saha and Zuber correlation에 대한 St, Xosv 계산이 모두 끝났습니다.")
elif m_idx == 2:
    nm = 'psz'
    for i, row in prop_tb.iterrows():
        prop_tb.loc[i, 'dt_psz'], prop_tb.loc[i, 'x_psz'] = mod.cal_PSZ(prop_tb.loc[i, 'q'], prop_tb.loc[i, 'rhof'], prop_tb.loc[i, 'dh'], prop_tb.loc[i, 'g'],
                                               prop_tb.loc[i, 'cpf'], prop_tb.loc[i, 'kf'], prop_tb.loc[i, 'pe'], prop_tb.loc[i, 'lam'])
        prop_tb.loc[i,'st_cal'] = prop_tb.loc[i,'q'] * 10 ** 6 / (prop_tb.loc[i,'g'] * prop_tb.loc[i,'cpf'] * prop_tb.loc[i,'dt_psz'])
    print("Park, Saha and Zuber correlation에 대한 St, Xosv 계산이 모두 끝났습니다.")
elif m_idx == 3:
    nm = 'unal'
    for i, row in prop_tb.iterrows():
        prop_tb.loc[i, 'dt_unal'], prop_tb.loc[i, 'x_unal'] = mod.cal_Unal(prop_tb.loc[i, 'q'], prop_tb.loc[i, 'pr'], prop_tb.loc[i, 'dh'], prop_tb.loc[i, 'v'],
                                            prop_tb.loc[i, 'cpf'], prop_tb.loc[i, 'kf'], prop_tb.loc[i, 're'], prop_tb.loc[i, 'refri'], prop_tb.loc[i, 'lam'])
        prop_tb.loc[i,'st_cal'] = prop_tb.loc[i,'q'] * 10 ** 6 / (prop_tb.loc[i,'g'] * prop_tb.loc[i,'cpf'] * prop_tb.loc[i,'dt_unal'])
    print("Unal correlation에 대한 St, Xosv 계산이 모두 끝났습니다.")
elif m_idx == 4:
    nm = 'levy'
    for i, row in prop_tb.iterrows():
        prop_tb.loc[i, 'dt_levy'], prop_tb.loc[i, 'x_levy'] = mod.cal_Levy(prop_tb.loc[i, 'sigma'], prop_tb.loc[i, 'dh'], prop_tb.loc[i, 'rhof'], prop_tb.loc[i, 'muf'],
                                            prop_tb.loc[i, 'kf'], prop_tb.loc[i, 're'], prop_tb.loc[i, 'pr'], prop_tb.loc[i, 'cpf'],
                                            prop_tb.loc[i, 'g'], prop_tb.loc[i, 'q'], prop_tb.loc[i, 'lam'], prop_tb.loc[i, 'v'])
        prop_tb.loc[i,'st_cal'] = prop_tb.loc[i,'q']* 10 ** 6 / (prop_tb.loc[i,'g'] * prop_tb.loc[i,'cpf'] * prop_tb.loc[i,'dt_levy'])
    print("Levy Model에 대한 St, Xosv 계산이 모두 끝났습니다.")
elif m_idx == 5:
    nm = 'bowr'
    for i, row in prop_tb.iterrows():
        prop_tb.loc[i, 'dt_bowr'], prop_tb.loc[i, 'x_bowr'] = mod.cal_Bowr(prop_tb.loc[i, 'p'], prop_tb.loc[i, 'q'], prop_tb.loc[i, 'v'], prop_tb.loc[i, 'lam'],
                                                  prop_tb.loc[i, 'cpf'])
        prop_tb.loc[i,'st_cal'] = prop_tb.loc[i,'q'] * 10 ** 6 / (prop_tb.loc[i,'g'] * prop_tb.loc[i,'cpf'] * prop_tb.loc[i,'dt_bowr'])
    print("Bowring correlation에 대한 St, Xosv 계산이 모두 끝났습니다.")
else:
    nm = 'js'
    for i, row in prop_tb.iterrows(): # geo, q, dh, dl, v, rhof, rhov, muf, muv, cpf, kf, lam, sigma, Pe, Pr
        prop_tb.loc[i, 'dt_js'], prop_tb.loc[i, 'x_js'], prop_tb.loc[i, 'we_js'] = mod.cal_Jeong(prop_tb.loc[i, 'geo'], prop_tb.loc[i, 'q'], prop_tb.loc[i, 'dh'], prop_tb.loc[i, 'dl'], prop_tb.loc[i, 'v'], prop_tb.loc[i, 'rhof'], prop_tb.loc[i, 'rhov'], prop_tb.loc[i, 'muf'], prop_tb.loc[i, 'muv'],prop_tb.loc[i, 'cpf'],prop_tb.loc[i, 'kf'],prop_tb.loc[i, 'lam'],prop_tb.loc[i, 'sigma'],prop_tb.loc[i, 'pe'],prop_tb.loc[i, 'pr'])
        prop_tb.loc[i,'st_cal'] = prop_tb.loc[i,'q'] * 10 ** 6 / (prop_tb.loc[i,'g'] * prop_tb.loc[i,'cpf'] * prop_tb.loc[i,'dt_js'])
    print("Jeong and Shim correlation에 대한 St, Xosv 계산이 모두 끝났습니다.")

print('END TIME of INITIALIZATION : ',str(datetime.now())[10:19])

In [ ]:
#val_chf_deng_tb = pd.concat([prop_tb, res_tb], axis=1) # column bind
#val_chf_park_tb = pd.concat([prop_tb, res_tb], axis=1) # column bind

In [ ]:
prop_tb.columns

In [ ]:
def calCHFDeng(self, rdcp=0.1, dh=0.1, g=0.1, xt_cal=0.1):
        """
        Deng (1997) CHF correlation
        """
        # CHF 계산 (Deng)
        alpha = round(1.669-6.544*(rdcp-0.448)**2,16)
        gamma = round(0.06523 + (0.1045/(np.sqrt((2*np.pi)*(np.log(rdcp))**2))) * np.exp(-5.413*((np.log(rdcp)+0.4537)**2/(np.log(rdcp)**2))),16)
        zxt = round((g*xt_cal*((1+xt_cal**2)**3))**0.5,16)
        q_cal = round((alpha/math.sqrt(dh)) * math.exp(-gamma*zxt),16) # Park
        return alpha, gamma, zxt, q_cal

In [ ]:
#for i in prop_tb.index:
global j
j = 0
tolerance = 0.0001
log_tb = pd.DataFrame()

for i in range(0, 10):
    cnt = 0
    # Preparing alpha and gamma
    prop_tb.loc[i, 'alpha'] = round(1.669-6.544*(prop_tb.loc[i, 'rdcp']-0.448)**2,16)
    prop_tb.loc[i, 'gamma'] = round(0.06523 + (0.1045/(np.sqrt((2*np.pi)*(np.log(prop_tb.loc[i, 'rdcp']))**2))) * np.exp(-5.413*((np.log(prop_tb.loc[i, 'rdcp'])+0.4537)**2/(np.log(prop_tb.loc[i, 'rdcp'])**2))),16)

    while 1:
        # Prepare : log-table
        log_tb.loc[j, 'run_id'] = int(i)
        log_tb.loc[j, 'org_xi'] = prop_tb.loc[i, 'xi']
        log_tb.loc[j, 'org_xe'] = prop_tb.loc[i, 'xe']
        log_tb.loc[j, 'org_q'] = prop_tb.loc[i, 'q']
        #log_tb.loc[j, 'xosv'] = prop_tb.loc[i, 'xosv']

        if j == 0: # initializaton step
            # Step 0: Preparing old values
            log_tb.loc[j, 'old_xe'] = prop_tb.loc[i, 'xe']
            log_tb.loc[j, 'old_qcal'] = prop_tb.loc[i, 'q'] # initilization of old_qcal = experimental qCHF

            # Step 1: Calculate old_xosv (initailzation using experimental qCHF)
            log_tb.loc[j, 'old_dtsz'], log_tb.loc[j, 'old_xosv'] = mod.cal_SZ(log_tb.loc[j, 'old_qcal'], prop_tb.loc[i, 'rhof'], prop_tb.loc[i, 'dh'], prop_tb.loc[i, 'g'], 
                                               prop_tb.loc[i, 'cpf'], prop_tb.loc[i, 'kf'], prop_tb.loc[i, 'pe'], prop_tb.loc[i, 'lam']/1e-3)
        else:
            pass
        # Step 2: Caluclate old_xt (initialization)
        log_tb.loc[j, 'old_xt'] = round(mod.cal_xt(log_tb.loc[j, 'org_xi'], log_tb.loc[j, 'old_xosv'], log_tb.loc[j, 'old_xe']),4)        
            #m = (1/g)*((1/gamma)*math.log((q*dh**0.5)/alpha)**(2/3)) # need to calculate old_Xt steps
            #log_tb.loc[j, 'old_xt'] = 0.1 # temporary value
            #print(prop_tb.loc[i, 'xt'])

        # Step 3: Calculate new_xe
        log_tb.loc[j, 'new_xe'] = pro.cal_xe(log_tb.loc[j, 'old_qcal'], prop_tb.loc[i, 'doi'], prop_tb.loc[i, 'dio'], prop_tb.loc[i, 'geo'], prop_tb.loc[i, 'hs'], prop_tb.loc[i, 'g'], prop_tb.loc[i, 'enthin'], prop_tb.loc[i, 'lh'], prop_tb.loc[i, 'lam'])

        # Step 4: Calculate new_xt using rate equation
        log_tb.loc[j, 'new_xt'] = round(mod.cal_xt(log_tb.loc[j, 'org_xi'], log_tb.loc[j, 'old_xosv'], log_tb.loc[j, 'new_xe']),4)


        # Step 5: Calculate new_qCHF
        log_tb.loc[j, 'alpha'], log_tb.loc[j, 'gamma'], log_tb.loc[j, 'zxt'], log_tb.loc[j, 'new_qcal'] = calCHFDeng(prop_tb.loc[i, 'rdcp'], prop_tb.loc[i, 'dh'], prop_tb.loc[i, 'g'], log_tb.loc[j, 'new_xt'])
        print("qexp: {:.2f}, qcal: {:.2f}".format(prop_tb.loc[i, 'q'], prop_tb.loc[i, 'new_qcal']))
        print("old_xt: {:.2f}, new_xt: {:.2f}".format(log_tb.loc[j, 'old_xt'], log_tb.loc[j, 'new_xt']))


        # Step 7: Loop control
        #j += 1 # increase cnt of row number of log-table
        cnt += 1 # Increase limitation
        
        # Step 5: Valudation
        val = round(abs((log_tb.loc[j, 'old_xt'] - log_tb.loc[j, 'new_xt'])/log_tb.loc[j, 'new_xt']),6)
        print("val: {:.6f}".format(val))
        
        if val <= tolerance:
            print("Data converged")
            print(j)
            break
        else:
            if cnt > 100:
                print("This data doesn't converged")
                del cnt
                break
            else:
                # Step 5: Recurrsive state
                log_tb.loc[j+1, 'old_xt'] = (log_tb.loc[j, 'old_xt'] + 1)/2
                log_tb.loc[j+1, 'old_qcal'] = log_tb.loc[j, 'new_qcal']
                j += 1
                continue


In [ ]:
#prop_tb.loc[979, ['xi', 'xe', 'xt']]
log_tb

In [ ]:
prop_tb[prop_tb.loc[:, 'xt'] < 0].to_markdown

In [ ]:
# CHF알고리즘
# Validation set와  error table 만드는 작업 진행
val_chf_deng_tb = res_tb.loc[:,['p', 'dh','lh', 'q', 'g', 'xi', 'xe', 'xe', 'st_cal', 'lam', 'rdcp']]
val_chf_park_tb = res_tb.loc[:,['p', 'dh','lh', 'q', 'g', 'xi', 'xe', 'xe', 'st_cal', 'lam', 'rdcp']]

# Diverged data의 Xt를 추적하기 위한 테이블
#val_chf_xt_tb = pd.DataFrame()
#val_chf_xt_tb.loc[:, 'data_id'] = ""
#val_chf_xt_tb.loc[:, 'cnt_tr'] = ""

# 초기 tolerance 설정
stepsize = 1e-6
tolerance = 1e-10
init_xt = 1e-6

# ----------------------------
# 시작부분 코드
start_time = time.time()
print("Start calucation of Xt convergence test")


for i in val_chf_deng_tb.index:
    xt_cal_old = 0
    if cal_idx == 4:
        # CHF 모델별로 계산
        val_chf_deng_tb.loc[i, 'xosv_cal'], val_chf_deng_tb.loc[i, 'xeq'], val_chf_deng_tb.loc[i, 'q_cal'], val_chf_deng_tb.loc[i, 'xt_cal'], val_chf_deng_tb.loc[i, 'alpha'], val_chf_deng_tb.loc[i, 'gamma'], val_chf_deng_tb.loc[i, 'Fxt'], val_chf_deng_tb.loc[i, 'convergence'] = mod.calAlgCHFKim(i, prop_tb.loc[i, 'rdcp'], raw_tb.loc[i, 'dh'], raw_tb.loc[i, 'lh'], raw_tb.loc[i, 'g'], raw_tb.loc[i, 'q'], val_chf_deng_tb.loc[i, 'xi'], prop_tb.loc[i, 'xe'], xt_cal_old, val_chf_deng_tb.loc[i, 'st_cal'], prop_tb.loc[i, 'lam'], modCHF = 'Deng', stepsize = stepsize, tolerance = tolerance) # Deng model
        
        # 나중에 한꺼번에 게산할 수 있도록 개선
        val_chf_deng_tb.loc[:, 'Zxt'] = np.sqrt(val_chf_deng_tb.loc[:,'g']*val_chf_deng_tb.loc[:,'xt_cal']*(1+val_chf_deng_tb.loc[:,'xt_cal']**2)**3) # Zxt 계산
        val_chf_deng_tb.loc[:, 'alpha_cal'] = (val_chf_deng_tb.loc[:, 'q_cal'] * np.sqrt(val_chf_deng_tb.loc[:, 'dh'])) / (np.exp(-val_chf_deng_tb.loc[:, 'gamma']*val_chf_deng_tb.loc[:, 'Zxt'])) # alpha_cal 계산
        val_chf_deng_tb.loc[:, 'gamma_cal'] = -np.log(val_chf_deng_tb.loc[:, 'q_cal'] * np.sqrt(val_chf_deng_tb.loc[:, 'dh']) / (val_chf_deng_tb.loc[:, 'alpha'])) / val_chf_deng_tb.loc[:, 'Zxt'] # gamma_cal 계산

        # Park correlation
        val_chf_park_tb.loc[i, 'xosv_cal'], val_chf_park_tb.loc[i, 'xeq'], val_chf_park_tb.loc[i, 'q_cal'], val_chf_park_tb.loc[i, 'xt_cal'], val_chf_park_tb.loc[i, 'alpha'], val_chf_park_tb.loc[i, 'gamma'], val_chf_park_tb.loc[i, 'k1'], val_chf_park_tb.loc[i, 'k2'], val_chf_park_tb.loc[i, 'k3'], val_chf_park_tb.loc[i, 'Fxt'], val_chf_park_tb.loc[i, 'convergence'] = mod.calAlgCHFKim(i, prop_tb.loc[i, 'rdcp'], raw_tb.loc[i, 'dh'], raw_tb.loc[i, 'lh'], raw_tb.loc[i, 'g'], raw_tb.loc[i, 'q'], val_chf_park_tb.loc[i, 'xi'], prop_tb.loc[i, 'xe'], xt_cal_old, val_chf_park_tb.loc[i, 'st_cal'], prop_tb.loc[i, 'lam'], modCHF = 'Park', stepsize = stepsize, tolerance = tolerance) # Park model

        # 나중에 한꺼번에 계산할 수 있도록 개선
        val_chf_park_tb.loc[:, 'Zxt'] = np.sqrt(val_chf_park_tb.loc[:,'g']*val_chf_park_tb.loc[:,'xt_cal']*(1+val_chf_park_tb.loc[:,'xt_cal']**2)**3) # Zxt 계산
        val_chf_park_tb.loc[:, 'alpha_cal'] = (val_chf_park_tb.loc[:, 'q_cal'] * np.sqrt(val_chf_park_tb.loc[:, 'dh'])) / (np.exp(-val_chf_park_tb.loc[:, 'gamma']*val_chf_park_tb.loc[:, 'Zxt'])) # alpha_cal 계산
        val_chf_park_tb.loc[:, 'gamma_cal'] = -np.log(val_chf_park_tb.loc[:, 'q_cal'] * np.sqrt(val_chf_park_tb.loc[:, 'dh']) / (val_chf_park_tb.loc[:, 'alpha'])) / val_chf_park_tb.loc[:, 'Zxt'] # gamma_cal 계산

        # Error 값 계산
        val_chf_deng_tb.loc[i, 'q_err'] = round((val_chf_deng_tb.loc[i,'q_cal']/val_chf_deng_tb.loc[i,'q']-1),12)
        val_chf_park_tb.loc[i, 'q_err'] = round((val_chf_park_tb.loc[i,'q_cal']/val_chf_park_tb.loc[i,'q']-1),12)
    else:
        # CHF 모델별로 계산
        val_chf_deng_tb.loc[i, 'xosv_cal'], val_chf_deng_tb.loc[i, 'xeq'], val_chf_deng_tb.loc[i, 'q_cal'], val_chf_deng_tb.loc[i, 'xt_cal'], val_chf_deng_tb.loc[i, 'alpha'], val_chf_deng_tb.loc[i, 'gamma'], val_chf_deng_tb.loc[i, 'Fxt'], val_chf_deng_tb.loc[i, 'convergence'] = mod.calAlgCHFPark(i, prop_tb.loc[i, 'rdcp'], raw_tb.loc[i, 'dh'], raw_tb.loc[i, 'lh'], raw_tb.loc[i, 'g'], raw_tb.loc[i, 'q'], val_chf_deng_tb.loc[i, 'xi'], prop_tb.loc[i, 'xe'],  xt_cal_old, val_chf_deng_tb.loc[i, 'st_cal'], prop_tb.loc[i, 'lam'], modCHF = 'Deng', stepsize = stepsize, tolerance = tolerance) # Deng model
        
        # 나중에 한꺼번에 게산할 수 있도록 개선
        val_chf_deng_tb.loc[:, 'Zxt'] = np.sqrt(val_chf_deng_tb.loc[:,'g']*val_chf_deng_tb.loc[:,'xt_cal']*(1+val_chf_deng_tb.loc[:,'xt_cal']**2)**3) # Zxt 계산
        val_chf_deng_tb.loc[:, 'alpha_cal'] = (val_chf_deng_tb.loc[:, 'q_cal'] * np.sqrt(val_chf_deng_tb.loc[:, 'dh'])) / (np.exp(-val_chf_deng_tb.loc[:, 'gamma']*val_chf_deng_tb.loc[:, 'Zxt'])) # alpha_cal 계산
        val_chf_deng_tb.loc[:, 'gamma_cal'] = -np.log(val_chf_deng_tb.loc[:, 'q_cal'] * np.sqrt(val_chf_deng_tb.loc[:, 'dh']) / (val_chf_deng_tb.loc[:, 'alpha'])) / val_chf_deng_tb.loc[:, 'Zxt'] # gamma_cal 계산

        # Park correlation
        val_chf_park_tb.loc[i, 'xosv_cal'], val_chf_park_tb.loc[i, 'xeq'], val_chf_park_tb.loc[i, 'q_cal'], val_chf_park_tb.loc[i, 'xt_cal'], val_chf_park_tb.loc[i, 'alpha'], val_chf_park_tb.loc[i, 'gamma'], val_chf_park_tb.loc[i, 'k1'], val_chf_park_tb.loc[i, 'k2'], val_chf_park_tb.loc[i, 'k3'], val_chf_park_tb.loc[i, 'Fxt'], val_chf_park_tb.loc[i, 'convergence'] = mod.calAlgCHFPark(i, prop_tb.loc[i, 'rdcp'], raw_tb.loc[i, 'dh'], raw_tb.loc[i, 'lh'], raw_tb.loc[i, 'g'], raw_tb.loc[i, 'q'], val_chf_park_tb.loc[i, 'xi'], prop_tb.loc[i, 'xe'], xt_cal_old, val_chf_park_tb.loc[i, 'st_cal'], prop_tb.loc[i, 'lam'], modCHF = 'Park', stepsize = stepsize, tolerance = tolerance) # Park model

        # 나중에 한꺼번에 게산할 수 있도록 개선
        val_chf_park_tb.loc[:, 'Zxt'] = np.sqrt(val_chf_park_tb.loc[:,'g']*val_chf_park_tb.loc[:,'xt_cal']*(1+val_chf_park_tb.loc[:,'xt_cal']**2)**3) # Zxt 계산
        val_chf_park_tb.loc[:, 'alpha_cal'] = (val_chf_park_tb.loc[:, 'q_cal'] * np.sqrt(val_chf_park_tb.loc[:, 'dh'])) / (np.exp(-val_chf_park_tb.loc[:, 'gamma']*val_chf_park_tb.loc[:, 'Zxt'])) # alpha_cal 계산
        val_chf_park_tb.loc[:, 'gamma_cal'] = -np.log(val_chf_park_tb.loc[:, 'q_cal'] * np.sqrt(val_chf_park_tb.loc[:, 'dh']) / (val_chf_park_tb.loc[:, 'alpha'])) / val_chf_park_tb.loc[:, 'Zxt'] # gamma_cal 계산

        # Error 값 계산
        val_chf_deng_tb.loc[i, 'q_err'] = round((val_chf_deng_tb.loc[i,'q_cal']/val_chf_deng_tb.loc[i,'q']-1),12)
        val_chf_park_tb.loc[i, 'q_err'] = round((val_chf_park_tb.loc[i,'q_cal']/val_chf_park_tb.loc[i,'q']-1),12)

        # Error 값 계산
        val_chf_deng_tb.loc[i, 'q_err'] = round((val_chf_deng_tb.loc[i,'q_cal']/val_chf_deng_tb.loc[i,'q']-1),12)
        val_chf_park_tb.loc[i, 'q_err'] = round((val_chf_park_tb.loc[i,'q_cal']/val_chf_park_tb.loc[i,'q']-1),12)

# ----------------------------
# 종료부분 코드
print("start_time", start_time)
print("--- {} minutes ---".format(round(time.time() - start_time, 4)/60))

In [ ]:
# m_idx에 따른 이름 재생성
if m_idx == 1:
    m_idx_nm = 'sz'
elif m_idx == 2:
    m_idx_nm = 'msz'
elif m_idx == 3:
    m_idx_nm = 'unal'
elif m_idx == 4:
    m_idx_nm = 'levy'
elif m_idx == 5:
    m_idx_nm = 'bowr'
else:
    m_idx_nm = 'new'

# cal_idx에 따른 이름 재생성
if cal_idx == 1:
    cal_idx_nm = 'gs'
else:
    cal_idx_nm = 'bis'

# 데이터를 PostgreSQL로 저장하기
sql.write_sql(res_tb, 'chf_prop_tb',db_engine)
sql.write_sql(val_chf_deng_tb, 'chf_osv_' + m_idx_nm + '_alg_' + cal_idx_nm + '_deng_tb',db_engine)
sql.write_sql(val_chf_park_tb, 'chf_osv_' + m_idx_nm + '_alg_' + cal_idx_nm + '_park_tb',db_engine)

In [ ]:
def delOutlierZ(data, threshold = 3):
    """
    data = list
    thresshold = 표준편차 제한
    """
    mean = np.mean(data)
    std = np.std(data)
    z_score = [(y-mean) / std for y in data]
    mask = np.where(np.abs(z_score) < threshold)
    return mask # mask 필터 형태의 값 반환 

# Alpha calcuation function
# Deng model을 기준으로 alpha, gamma를 계산하는 알고리즘.
# q ~ Xt를 비선형 회귀 (같은 p에서)

In [ ]:
# alpha, gamma 갱신 알고리즘
# Rdcp 테이블 작성
df_rdcp = pd.DataFrame(columns=['rdcp','alpha_avg', 'gamma_avg'])

for i, row in enumerate(res_tb.rdcp.unique()):
    # 같은 압력 조건 샘플
    mask1 = (val_chf_deng_tb.rdcp == row)
    smp_tb = val_chf_deng_tb.loc[mask1, ['rdcp','alpha_cal','gamma_cal']]
    #mask2 = delOutlierZ(tt_tb.alpha_cal, threshold=1)[0] # 이상치 제거하는 알고리즘 추가 필요
    # alpha_cal 평균 계산
    df_rdcp.loc[i,'rdcp'] = row
    df_rdcp.loc[i,'alpha_avg'] = round(np.mean(smp_tb.alpha_cal),16)
    df_rdcp.loc[i,'gamma_avg'] = round(np.mean(smp_tb.gamma_cal),16)
    if df_rdcp.loc[i,'gamma_avg'] == np.inf:
        df_rdcp.loc[i, 'gamma_avg'] = np.nan
    else:
        pass
    del smp_tb

df_rdcp = df_rdcp.dropna() # inf 뜰 경우 제외

In [ ]:
# alpha_avg 다항회귀
X = np.c_[df_rdcp.loc[:,'rdcp'], (df_rdcp.loc[:,'rdcp'])**2]
y1 = df_rdcp.loc[:,'alpha_avg']
y2 = df_rdcp.loc[:, 'gamma_avg']

# 선형 모델
model1 = LinearRegression()
model1.fit(X, y1)
model2 = LinearRegression()
model2.fit(X, y2)

xs = np.arange(min(df_rdcp.rdcp),max(df_rdcp.rdcp),0.02) # x 범위의 순차값 생성
ys1 = xs*model1.coef_[0] + (xs**2)*model1.coef_[1] + model1.intercept_
ys2 = xs*model2.coef_[0] + (xs**2)*model2.coef_[1] + model2.intercept_
# 2차 다항식 회귀

# 그래프
fig = plt.figure()
ax1 = fig.add_subplot(2, 2, 1)
ax2 = fig.add_subplot(2, 2, 2)

plt.xlabel('Reduced Pressure [-]')
plt.ylabel('Alpha or Gamma')
plt.legend(loc='upper left')
ax1.scatter(df_rdcp.loc[:,'rdcp'], df_rdcp.loc[:,'alpha_avg'], color = 'black', label = 'Alpha', alpha=1)
ax2.scatter(df_rdcp.loc[:,'rdcp'], df_rdcp.loc[:,'gamma_avg'], color = 'blue', label = 'Gamma', alpha=1)

ax1.plot(xs,ys1,'r-',lw=3)
ax2.plot(xs,ys2,'b-',lw=3)


print(round(model1.coef_[0],4), round(model1.coef_[1],4), round(model1.intercept_,4))
print(round(model2.coef_[0],4), round(model2.coef_[1],4), round(model2.intercept_,4))